In [1]:
import os
import numpy as np
import pandas as pd
import cv2
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import torch 
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader, Dataset

/Users/adirserruya/Desktop/breast_cancer_classification/classification_venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Importing Images dataset
from glob import glob
imagePatches = glob('/Users/adirserruya/Desktop/breast_cancer_classification/IDC_regular_ps50_idx5/**/**/*.png', recursive=True)
len(imagePatches) # How many images do we have ?

832572

In [3]:
imagePatches[0] # Checkout how the images are notated

'/Users/adirserruya/Desktop/breast_cancer_classification/IDC_regular_ps50_idx5/9036/0/9036_idx5_x1051_y2401_class0.png'

In [4]:
import fnmatch
patternZero = '*class0.png' # What ever image that ends with class 0 
patternOne = '*class1.png'# What ever image that ends with class 1
classZero = fnmatch.filter(imagePatches, patternZero) #Unix filename pattern matching returns True or False if ImagePatches match pattern Zero
classOne = fnmatch.filter(imagePatches, patternOne)

In [5]:
#Creating the labels vector takes 49minutes to run on local machine CPU M1
y = []
for img in imagePatches:
    if img in classZero:
        y.append(0)
    elif img in classOne:
        y.append(1)

In [6]:
#Creating a Data frame with columns images|labels so per image we will know the class
images_df = pd.DataFrame()
images_df["images"] = imagePatches
images_df["labels"] = y

In [7]:
#view samples of the DF
images_df.head()

,images,labels
0,/Users/adirserruya/Desktop/breast_cancer_class...,0
1,/Users/adirserruya/Desktop/breast_cancer_class...,0
2,/Users/adirserruya/Desktop/breast_cancer_class...,0
3,/Users/adirserruya/Desktop/breast_cancer_class...,0
4,/Users/adirserruya/Desktop/breast_cancer_class...,0


In [8]:
#Count how many images we have of each label
images_df.groupby("labels")["labels"].count()

labels
0    596214
1    236358
Name: labels, dtype: int64

In [9]:
#Splitting data into train and val sets
train, val = train_test_split(images_df, stratify=images_df.labels, test_size=0.3)
len(train), len(val)

(582800, 249772)

In [ ]:
## Comment for my self - Data is unbalanced (too many samples of label 0 comared to samples with label 1 need to 
# to something about it 
# Options : 
# 1) Sample Weights
# 2) Generate Augmented Data only in the minority Class
# 3) Resampling Technique - A widely adopted technique for dealing with highly unbalanced datasets is 
# called resampling. It consists of removing samples from the majority class (under-sampling) 
# and/or adding more examples from the minority class (over-sampling).
# 4) Penalize Learning Algorithm - 

In [10]:
#Create custom dataset class (Inheritance from Dataset)
class MyDataset(Dataset):
    def __init__(self, df_data,transform=None):
        super().__init__()
        self.df = df_data.values
        self.transform = transform

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_path,label = self.df[index]
        image = cv2.imread(img_path)#Reading image
        image = cv2.resize(image, (50,50))#resizing 50x50
        if self.transform is not None:
            image = self.transform(image)
        return image, label

In [11]:
## Parameters for model

# Hyper parameters
num_epochs = 1 ## On purpose only 1 epoch - because my pc take 16.7 days to run 8 epochs.
num_classes = 2
batch_size = 128
learning_rate = 0.002

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [12]:
#Train transformation
trans_train = transforms.Compose([transforms.ToPILImage(),
                                  transforms.Pad(64, padding_mode='reflect'),##Turning 50x50 to (64+50+64)x(64+50+64)=178x178x3 <- Input shape
                                  transforms.RandomHorizontalFlip(),## Whether Malignant or Benign Direction does not matter so HFlip Data Augmentation to make model more robust
                                  transforms.RandomVerticalFlip(),## Same as above
                                  transforms.RandomRotation(20),#More Data Augmentation 
                                  #random.dropout()(Patches)
                                  #downgrading image quality  
                                  transforms.ToTensor(), ## Converting PIL To Tensor
                                  transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5])])## Normalizing every channel ( Has to be a Tensor )

trans_valid = transforms.Compose([transforms.ToPILImage(),
                                  transforms.Pad(64, padding_mode='reflect'),
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5])])

#Creating the actual dataset to be learned from after augmentation
dataset_train = MyDataset(df_data=train, transform=trans_train)
dataset_valid = MyDataset(df_data=val,transform=trans_valid)
#Creating the date loader objects
loader_train = DataLoader(dataset = dataset_train, batch_size=batch_size, shuffle=True, num_workers=0)
loader_valid = DataLoader(dataset = dataset_valid, batch_size=batch_size//2, shuffle=False, num_workers=0)

In [13]:
dataset_train[0][0].shape ## Input size shape after padding 

torch.Size([3, 178, 178])

In [139]:
#Creating the model according to planned architecture
class SimpleCNN(nn.Module):
    def __init__(self):
        # ancestor constructor call
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=2) 
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=2)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=2)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=2)
        self.conv5 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=2)

        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)
        self.bn4 = nn.BatchNorm2d(256)
        self.bn5 = nn.BatchNorm2d(512)

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.avg = nn.AvgPool2d(7)
        self.fc = nn.Linear(512 * 1 * 1, 2) # !!!
        
    def forward(self, x):
        #Functions Occour Inside Out self.conv1-->self.bn1-->F.leaky_relu-->pool
        
        x = self.pool(F.leaky_relu(self.bn1(self.conv1(x)))) # first convolutional layer then batch-norm, then activation then pooling layer.
        x = self.pool(F.leaky_relu(self.bn2(self.conv2(x))))
        x = self.pool(F.leaky_relu(self.bn3(self.conv3(x))))
        x = self.pool(F.leaky_relu(self.bn4(self.conv4(x))))
        x = self.pool(F.leaky_relu(self.bn5(self.conv5(x))))
        x = self.avg(x)

        #print(x.shape) # lifehack to find out the correct dimension for the Linear Layer (!!! IMPORTANT !!!)
        x = x.view(-1, 512 * 1 * 1) # !!!
        x = self.fc(x)
        return x

In [140]:
## GPU (in case i had one) ,so CPU for me.
model = SimpleCNN().to(device)

In [141]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss() ## When using cross entropy-loss then there's no need for softmax
optimizer = torch.optim.Adamax(model.parameters(), lr=learning_rate)

In [ ]:
# Train the model
total_step = len(loader_train)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(loader_train):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad() ## SUPER IMPORTANT TO Zero out the gradient
        loss.backward() ## Propogate backwards the gradient
        optimizer.step() ## Change weights according to the opposite direction of the gradient.
        #Print Progress
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

In [ ]:
# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
confusion_matrix = torch.zeros(2, 2)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in loader_valid:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        for t, p in zip(labels.view(-1), predicted.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1
                 
    print('Test Accuracy of the model on the test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

In [ ]:
print(confusion_matrix)